<a href="https://colab.research.google.com/github/indavarapuaneesh/mission_intern/blob/master/fastai_8_foundations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import operator

In [0]:
def test(a, b, cmp, cname=None): #for comparing objects
    if cname is None : cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"
        
def test_eq(a, b): test(a, b, operator.eq, '==')

In [0]:
# test_eq(TEST, 'test')

In [0]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
path = datasets.download_data(MNIST_URL, ext='.gz')
path

PosixPath('/root/.fastai/data/mnist.pkl.gz')

In [0]:
with gzip.open(path, 'rb') as f :
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [0]:
x_train, y_train, x_valid, y_valid = map(tensor, (x_train, y_train, x_valid, y_valid))

n, c = x_train.shape
x_train, x_train.shape, y_train, y_train.shape, y_train.min(), y_train.max()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([50000, 784]),
 tensor([5, 0, 4,  ..., 8, 4, 8]),
 torch.Size([50000]),
 tensor(0),
 tensor(9))

In [0]:
def normalize(x, m, s): return (x-m)/s

In [0]:
train_mean, train_std = x_train.mean(), x_train.std()

train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std)

x_valid = normalize(x_valid, train_mean, train_std)

In [0]:
def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f"Near zero : {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [0]:
n,m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

In [0]:
nh = 50

In [0]:
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.randn(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [0]:
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
def lin(x, w, b): return x@w + b

In [0]:
t = lin(x_valid, w1, b1)

In [0]:
t.mean(), t.std()

(tensor(-0.0750), tensor(1.4519))

In [0]:
def relu(x) : return x.clamp_min(0.)

In [0]:
t = relu(lin(x_valid, w1, b1))

In [0]:
t.mean(), t.std()

(tensor(0.5358), tensor(0.8024))

In [0]:
w1 = torch.randn(m, nh) *math.sqrt(2/m)

In [0]:
w1.mean(), w1.std()

(tensor(0.0003), tensor(0.0504))

In [0]:
t = relu(lin(x_valid, w1, b1))

In [0]:
from torch.nn import init

In [0]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [0]:
init.kaiming_normal_??

In [0]:
w1.mean(), w1.std()

(tensor(0.0003), tensor(0.0506))

In [0]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [0]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [0]:
torch.nn.Linear.forward??

In [0]:
torch.nn.functional.linear??

In [0]:
torch.nn.Conv2d??

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
def relu(x) : return x.clamp_min(0.) - 0.5

In [0]:
w1 = torch.randn(m, nh) * math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))

In [0]:
t1.mean(), t1.std()

(tensor(0.1574), tensor(0.9225))

In [0]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [0]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 22.1 ms per loop


In [0]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

In [0]:
model(x_valid).shape

torch.Size([10000, 1])

In [0]:
def mse(output, target): return (output.squeeze(-1)-target).pow(2).mean()

In [0]:
y_train, y_valid = y_train.float(), y_valid.float()

In [0]:
preds = model(x_train)

In [0]:
preds.shape

torch.Size([50000, 1])

In [0]:
mse(preds, y_train)

tensor(30.7409)

In [0]:
def mse_grad(inp, target):
    inp.g = 2. * (inp.squeeze()-target).unsqueeze(-1)/inp.shape[0]
    print("gradients for the last layers output shape is  ", inp.g.shape)

In [0]:
def relu_grad(inp, out):
    inp.g = (inp>0).float() * out.g

In [0]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [0]:
def forward_and_backward(inp, target):
    l1 = inp @ w1 + b1
    # print(l1.shape) #passing through first layer
    l2 = relu(l1) #passing through activation unit
    # print(l2.shape)
    out = l2 @ w2 + b2 #for generating output
    # print(out.shape)
    # print(target.shape)
    loss = mse(out, target) #computing loss
    
    mse_grad(out, target) #computing gradients w.r.t output functions
    print(out.shape)
    print(target.shape)
    lin_grad(l2, out, w2, b2) 
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [0]:
forward_and_backward(x_train, y_train)

gradients for the last layers output shape is   torch.Size([50000, 1])
torch.Size([50000, 1])
torch.Size([50000])


In [0]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone() 

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, target):
  l1 = inp @ w12 + b12
  l2 = relu(l1)
  out = l2 @ w22 + b22

  return mse(out, target)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
class Relu():
  def __call__(self, inp):
    self.inp = inp
    self.out = inp.clamp_min(0.) - 0.5
    return self.out
  
  def backward(self):
    self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
  def __init__(self, w, b):
    self.w, self.b  = w, b
  
  def __call__(self, inp):
    self.inp = inp
    self.out = inp @ self.w + self.b

    return self.out

  def backward(self):
    self.inp.g = self.out.g @ self.w.t()
    self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
    self.b.g = setest_near_zero(w22.grad-w2g,0)lf.out.g.sum(0)

In [0]:
class Mse():
  def __call__(self, inp, target):
    self.inp = inp
    self.target = target
    self.out = (inp.squeeze()-target).pow(2).mean()
    return self.out
  
  def backward(self):
    self.inp.g = 2. * (self.inp.squeeze() - self.target).unsqueeze(-1) / self.target.shape[0]

In [0]:
class Model():
  def __init__(self, w1, b1, w2, b2):
    self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
    self.loss = Mse()

  def __call__(self, x, target):
    for l in self.layers: x = l(x)
    return self.loss(x, target)
  
  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers):
      l.backward()

In [0]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [0]:
%time loss = model(x_train, y_train)

CPU times: user 119 ms, sys: 9.66 ms, total: 129 ms
Wall time: 129 ms


In [0]:
%time model.backward()

CPU times: user 4.62 s, sys: 257 ms, total: 4.88 s
Wall time: 4.89 s


In [0]:
class Module():
  def __call__(self, *args):
    self.args = args
    self.out = self.forward(*args)
    return self.out
  
  def forward(self):
    raise Exception('not implemented')
  def backward(self):
    self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
  def forward(self, inp):
    return inp.clamp_min(0.) - 0.5
  def bwd(self, out, inp):
    inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
  def __init__(self, w, b): 
    self.w, self.b = w, b 
  
  def forward(self, inp):
    return inp @ self.w + self.b
  def bwd(self, out, inp):
    inp.g = out.g @ self.w.t()
    self.w.g = torch.einsum("bi,bj->ij",inp,out.g)
    self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
  def forward(self, inp, target):
    return (inp.squeeze()-target).pow(2).mean()
  def bwd(self, out, inp, target):
    inp.g = 2* (inp.squeeze()-target).unsqueeze(-1)/target.shape[0]

In [0]:
class Model():
  def __init__ (self):
    self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
    self.loss = Mse()
  
  def __call__(self, x, target):
    for l in self.layers:
      x = l(x)

    return self.loss(x, target)
  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers):
      l.backward()

In [0]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [0]:
%time loss = model(x_train, y_train)

CPU times: user 179 ms, sys: 0 ns, total: 179 ms
Wall time: 180 ms


In [0]:
%time model.backward()

CPU times: user 289 ms, sys: 7.36 ms, total: 296 ms
Wall time: 313 ms


In [0]:
from torch import nn

In [0]:
class Model(nn.Module):
  def __init__(self, n_in, nh, n_out):
    super().__init__()
    self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
    self.loss = mse
  def __call__(self, x, target):
    for l in self.layers: 
      x = l(x)
    
    return self.loss(x.squeeze(), target)

In [0]:
model = Model(m, nh, 1)

In [0]:
%time loss =model(x_train, y_train)

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 103 ms


In [0]:
%time loss.backward()

CPU times: user 77.5 ms, sys: 3.14 ms, total: 80.7 ms
Wall time: 86.1 ms
